In [2]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
!pip3 install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip3 install chromadb 


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip3 install jupyter --upgrade
!pip3 install langchain-community






Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import requests

/Users/apple/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
import chromadb
import numpy

In [8]:
res = requests.post('http://localhost:11434/api/embeddings',
                    json={
                        'model': 'llama3',
                        'prompt': 'Hello world' 
                    })

print(res.json())

{'embedding': [-3.0615127086639404, -2.040092706680298, 3.296297311782837, 0.5453862547874451, 0.8021398186683655, 1.7417690753936768, -0.3473708927631378, 1.804307460784912, -2.7268757820129395, 2.939164400100708, -0.3493029773235321, -1.2189806699752808, 0.567453145980835, -0.21763521432876587, -0.09458506852388382, 2.8936843872070312, 0.40455999970436096, 3.2239034175872803, -2.4429662227630615, 2.1748650074005127, -2.823455810546875, 1.3298728466033936, 1.357390284538269, 0.3917418122291565, -2.5523977279663086, 1.57057785987854, 2.1441190242767334, 8.368847846984863, 3.0083441734313965, -2.51922607421875, -1.2583260536193848, 5.575753688812256, 2.8378701210021973, 0.47998079657554626, 3.474661350250244, 1.7481316328048706, -3.9562864303588867, -0.520336925983429, -1.6449007987976074, 1.700429916381836, 0.924757182598114, -1.2721534967422485, -1.01193106174469, -1.8977446556091309, 2.8658316135406494, 2.167705535888672, -0.6363341808319092, 1.7653838396072388, 0.5697784423828125, -

In [9]:
dimension = len(res.json()['embedding']) # 4096

In [19]:
from PyPDF2 import PdfReader # used to extract text from pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter # split text in smaller snippets
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
import uuid


In [21]:
embeddings = (
    OllamaEmbeddings(model="nomic-embed-text")
) 

embedding = embeddings.embed_query("Hello World")
dimension = len(embedding)
print(len(embedding))

768


In [22]:
chroma_client = chromadb.Client()
collection2 = chroma_client.get_or_create_collection(name="test_chat_nomic1")


In [12]:
EXTRACTED_TEXT_FILE_PATH = "pdf_txt.txt" # text extracted from pdf
PDF_FILE_PATH = "shashwat_resume_070623.pdf"
CHUNK_SIZE = 100 # chunk size to create snippets
CHUNK_OVERLAP = 10 # check size to create overlap between snippets
CONFIDENCE_SCORE = 0.70 # specify confidence score to filter search results. [0,1] prefered: 0.75

In [23]:
def extract_text_from_pdf(file_path: str):
    # Open the PDF file using the specified file_path
    reader = PdfReader(file_path)
    # Get the total number of pages in the PDF
    number_of_pages = len(reader.pages)

    # Initialize an empty string to store extracted text
    pdf_text = ""

    # Loop through each page of the PDF
    for i in range(number_of_pages):
        # Get the i-th page
        page = reader.pages[i]
        # Extract text from the page and append it to pdf_text
        pdf_text += page.extract_text()
        # Add a newline after each page's text for readability
        pdf_text += "\n"
    
    # Specify the file path for the new text file
    file_path = EXTRACTED_TEXT_FILE_PATH

    # Write the content to the text file
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(pdf_text)


In [14]:
def get_uuid():
    return str(uuid.uuid4())

In [26]:
def create_embeddings(file_path: str):
      # Initialize a list to store text snippets
    snippets = []
    # Initialize a CharacterTextSplitter with specified settings
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    
    # Read the content of the file specified by file_path
    with open(file_path, "r", encoding="utf-8") as file:
            file_text = file.read()

    # Split the text into snippets using the specified settings
    snippets = text_splitter.split_text(file_text)
    print(len(snippets))

    x = numpy.zeros((len(snippets), dimension), dtype= 'float32')
    ids = []

    for i, snippet in enumerate(snippets):
       print(snippet)
       embedding = embeddings.embed_query(snippet)
#        res = requests.post('http://localhost:11434/api/embeddings',
#                     json={
#                         'model': 'llama3',
#                         'prompt': snippet 
#                     }) 
#        embedding = res.json()['embedding']
       ids.append(get_uuid())
       x[i] = numpy.array(embedding)

    collection2.add(embeddings = x,
            documents=snippets,
            ids=ids)

In [25]:
data=collection2.get(include=['embeddings', 'documents', 'metadatas'])
print(data['embeddings'])
print(data['ids'])
print(data['metadatas'])
print(data['documents'])



[]
[]
[]
[]


In [27]:
extract_text_from_pdf(PDF_FILE_PATH)
create_embeddings(EXTRACTED_TEXT_FILE_PATH)

33
Shashwat Srivastava Medium: https://medium.com/@shashwat12june Linkedin:
Linkedin: https://www.linkedin.com/in/shashwat-s-29583294/ Email: Shashwat12june@gmail.com Phone:
Phone: 9540086409Skills • Programming Languages: Ruby, C, Java, GO                       •
• Frameworks: Ruby on Rails • Database: MySQL, PSQL, Redis , Mongo
• Cloud Services : AWS, Firebase • Version Control Tools: Git
• Unit testing: Rspec • Tools: RubyMine, Android studio,
studio, Adobe Photoshop, Xcode, Jmeter, Postman, Jenkins • Interest: Data Structures, Algorithm
Algorithm Experience Tech Lead at JISR (Feb 2023 -present) •Collaborating with the marketplace team
team to enhance flexibility within the JISR ecosystem. Facilitating seamless integration through
through open APIs and webhooks for simplified connectivity. •Collaborating with the platform and
and DevOps teams to optimize infrastructure and establish enhanced logging and monitoring
capabilities.  Software Developer Engineer 2 at ZOMATO (March 2021 - 

In [128]:
def frame_response(results, ques):
   joined_string = "\n".join(results)
   prompt = joined_string + "\n Given this information, " + ques
   llm = Ollama(
      model="llama3"
   )
   return llm.invoke(prompt)

In [206]:
def answer_users_question(user_question):
    #query_text = "I work with jisr?"
    # res = requests.post('http://localhost:11434/api/embeddings',
    #                     json={
    #                         'model': 'llama3',
    #                         'prompt': user_question
    #                     })

    # embedding_arr = numpy.array([res.json()['embedding']], dtype='float32')
    embedding_arr = embeddings.embed_query(user_question)
    result = collection2.query(
        query_embeddings= embedding_arr,
        n_results=5
    )

    return frame_response(result['documents'][0], user_question)

In [205]:
# Start an infinite loop, allowing the user to ask questions
while True:
    
    # Prompt the user to input a question
    print("👤USER:")
    
    # Read the user's question from the console
    user_question = input("Enter question: ")
    print(user_question)
    
    # Print a separator for readability
    print("----------------------")
    
    # Check if the user entered an empty question
    if user_question =="exit":
        
        # If the user entered an empty question, exit the loop
        break
    else:
        
        # If the user entered a question, proceed to generate a response
        print("🤖 BOT:")
        
        # Call the function to generate an answer based on the user's question
        # and print the bot's response
        print(answer_users_question(user_question=user_question))
        
        # Print a separator for readability
        print("----------------------")

👤USER:
What is the work experience of shashwat?
----------------------
🤖 BOT:
Based on the provided information, Shashwat's work experience is:

1. Software Engineer at 1MG (no specific dates mentioned)
	* Worked on db optimisations, cost optimisations, and load testing.
2. Android Developer Intern at Payioo (August 2016 - March 2017)
3. Tech Lead at JISR (Feb 2023 - present)
	* Collaborating with the marketplace team to enhance flexibility within the JISR ecosystem
	* Facilitating seamless integration through algorithm development and database queries.

Note that there may be gaps in his work experience between these listed roles, but this information provides a summary of his known work experience.
----------------------
👤USER:
is there any mention of zomato?
----------------------
🤖 BOT:
Yes, according to the provided information, Shashwat Srivastava mentions that he was a Software Developer Engineer 2 at ZOMATO from March 2021 to January 2023.
----------------------
👤USER:
exit
---